In [ ]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/bc_trip259172515_230215.csv")
print(df.head()) 

C. Filtering

In [ ]:
#filtering using drop
df.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1)

In [ ]:
df

In [ ]:
#Filtering using usecols
columns_to_include = [0, 2, 3, 4, 5, 6, 7 ]
df = pd.read_csv("C:/Users/sriva/Downloads/bc_trip259172515_230215.csv", usecols=columns_to_include)

In [ ]:
df

In [ ]:
from datetime import datetime, timedelta
def timestamp(row):
    #get the date and time
    opd_date = datetime.strptime(row['OPD_DATE'], '%d%b%Y:%H:%M:%S')
    delta = timedelta(seconds=row['ACT_TIME'])
    return opd_date + delta

df['TIMESTAMP'] = df.apply(timestamp, axis=1)

print(df)

E.  More Filtering

In [ ]:
df1=df.drop(['OPD_DATE', 'ACT_TIME'], axis=1)
df1

In [ ]:
df1['dMETERS'] = df1['METERS'].diff()
df1['dTIMESTAMP'] = df1['TIMESTAMP'].diff()

#Calculating speed
df1['SPEED'] = df1.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'].total_seconds() if row['dTIMESTAMP'].total_seconds() != 0 else 0, axis=1)
print(df1)
# Drop unneeded columns
df2 = df1.drop(['dMETERS', 'dTIMESTAMP'], axis=1)

#Minimum, maximum and average speed
speed = df1['SPEED'].describe()
print(f"\n{df1['SPEED'].describe()}")
print("\nMinimum speed:", speed['min'])
print("Maximum speed:", speed['max'])
print("Average speed:", speed['mean'])

df2

G. Larger Data Set

In [ ]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/bc_veh4223_230215.csv")
print(df.head()) 

In [ ]:
#filtering using drop
df.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1)
df

In [ ]:
#Filtering using usecols
columns_to_include = [0, 2, 3, 4, 5, 6, 7 ]
df = pd.read_csv("C:/Users/sriva/Downloads/bc_veh4223_230215.csv", usecols=columns_to_include)
df

In [ ]:
df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'], format='%d%b%Y:%H:%M:%S')

group = df.groupby('EVENT_NO_TRIP')
dfs = []

for id, data in group:
    data = data.sort_values(by='ACT_TIME')
    
    data['TIMESTAMP'] = data.apply(lambda row: row['OPD_DATE'] + timedelta(seconds=row['ACT_TIME']), axis=1)
    data['dMETERS'] = data['METERS'].diff()
    data['dTIMESTAMP'] = data['TIMESTAMP'].diff()
    
    #Calculate the speed
    data['SPEED'] = data.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'].total_seconds() if row['dTIMESTAMP'].total_seconds() != 0 else 0, axis=1)
    print(f'\nTrip Number:{id}')
    speed = data['SPEED'].describe()
    print(f"{data['SPEED'].describe()}")
    print("\nMinimum speed:", speed['min'])
    print("Maximum speed:", speed['max'])
    print("Average speed:", speed['mean'])
    dfs.append(data)


df3 = pd.concat(dfs)
#drop the columns
df3 = df3.drop(['OPD_DATE', 'ACT_TIME','dMETERS', 'dTIMESTAMP'], axis=1)
print(df3)



In [ ]:
group2 = df3.groupby('VEHICLE_ID')

speed = []

for id, data in group2:
    # Filter data for '2023-02-15'
    data = data[data['TIMESTAMP'].dt.date == pd.to_datetime('2023-02-15').date()]
    if not data.empty:
        s = data['SPEED'].describe()
        speed.append({
            'VEHICLE_ID': id,
            'MIN_SPEED': s['min'],
            'MAX_SPEED': s['max'],
            'MEDIAN_SPEED': data['SPEED'].median()
        })

df4 = pd.DataFrame(speed)
print(df4)

In [ ]:
group3 = df3.groupby('VEHICLE_ID')

max = []

for vehicle_id, vehicle_data in group3:
    if not vehicle_data.empty:
        max_speed = vehicle_data['SPEED'].max()
        max_speed_rows = vehicle_data[vehicle_data['SPEED'] == max_speed].to_dict('records')
        for row in max_speed_rows:
            row['VEHICLE_ID'] = vehicle_id
            max.append(row)

df5 = pd.DataFrame(max)
df5 = df5[df5['TIMESTAMP'].dt.date == pd.to_datetime('2023-02-15').date()]
print(df5)